In [1]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.callbacks import CSVLogger
from tensorflow.python.client import device_lib

In [2]:
from keras.models import model_from_json
import matplotlib.pyplot as plt
import numpy as np
import os,random
from keras.preprocessing.image import img_to_array, load_img

In [3]:
device_lib.list_local_devices()
n_categories = 2
batch_size = 32
train_dir = './dataset/train'
validation_dir = './dataset/val'
file_name = 'vgg16_drink'

2022-01-24 03:45:51.845128: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-24 03:45:51.845338: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [4]:
base_model = VGG16(
    classes=2,
    weights = 'imagenet',
    include_top = False,
    input_tensor = Input(shape=(224, 224, 3)))

2022-01-24 03:46:07.596387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-24 03:46:07.596415: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction = Dense(n_categories, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)

In [6]:
for layer in base_model.layers[:15]: layer.trainable = False

In [8]:
model.compile(
    optimizer=SGD(learning_rate=0.0001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
train_datagen = ImageDataGenerator(rescale=1.0/255)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

validation_generator=validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

hist=model.fit_generator(train_generator,
                         epochs=100,
                         verbose=1,
                         validation_data=validation_generator,
                         callbacks=[CSVLogger(file_name+'.csv')])

model.save(file_name+'.h5')

Found 2559 images belonging to 2 classes.
Found 640 images belonging to 2 classes.
Epoch 1/100


/var/folders/t6/g40lkvln3ygcbm4lq1qqcp9h0000gn/T/ipykernel_43810/2224518313.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_generator,
2022-01-24 03:47:40.522461: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-24 03:47:40.711349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - ETA: 0s - loss: 0.6691 - accuracy: 0.6358

2022-01-24 03:48:34.973239: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - 66s 820ms/step - loss: 0.6691 - accuracy: 0.6358 - val_loss: 0.6477 - val_accuracy: 0.6687
Epoch 2/100
80/80 [==============================] - 66s 824ms/step - loss: 0.5716 - accuracy: 0.8312 - val_loss: 0.5670 - val_accuracy: 0.7547
Epoch 3/100
80/80 [==============================] - 69s 861ms/step - loss: 0.4547 - accuracy: 0.8898 - val_loss: 0.4757 - val_accuracy: 0.7719
Epoch 4/100
80/80 [==============================] - 69s 858ms/step - loss: 0.3405 - accuracy: 0.9082 - val_loss: 0.3908 - val_accuracy: 0.8188
Epoch 5/100
80/80 [==============================] - 70s 878ms/step - loss: 0.2594 - accuracy: 0.9281 - val_loss: 0.3382 - val_accuracy: 0.8281
Epoch 6/100
80/80 [==============================] - 69s 857ms/step - loss: 0.2077 - accuracy: 0.9383 - val_loss: 0.3018 - val_accuracy: 0.8469
Epoch 7/100
80/80 [==============================] - 70s 877ms/step - loss: 0.1674 - accuracy: 0.9523 - val_loss: 0.2637 - val_accuracy: 0.8641
Epoc

In [11]:
batch_size = 32
test_dir = './tst/'
display_dir = './display'
label = ['beer', 'wine']

model.load_weights(file_name+'.h5')

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

score = model.evaluate_generator(test_generator)
print('\n test loss:', score[0])
print('\n test_acc:', score[1])

files = os.listdir(display_dir)
img = random.sample(files, 4)

plt.figure(figsize=(4, 4))
for i in range(4):
    temp_img = load_img(os.path.join(display_dir, img[i]), target_size=(224, 224))
    plt.subplot(2, 2, i+1)
    plt.imshow(temp_img)

    temp_img_array = img_to_array(temp_img)
    temp_img_array = temp_img_array.astype('float32') / 255.0
    temp_img_array = temp_img_array.reshape((1, 224, 224, 3))

    img_pred = model.predict(temp_img_array)
    plt.title(label[np.argmax(img_pred)])

    plt.xticks([]),plt.yticks([])


plt.show()

/opt/homebrew/Caskroom/mambaforge/base/envs/ohya/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/var/folders/t6/g40lkvln3ygcbm4lq1qqcp9h0000gn/T/ipykernel_43810/1516596872.py:22: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(test_generator)


Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0